# 03. State, Action, and Tool Contract Design

Designing explicit state and action schemas is critical for reliable planning agents.


In [ ]:
from __future__ import annotations

import json
import os
import math
import random
import statistics
from pathlib import Path


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'README.md').exists() and (candidate / 'main_langgraph.py').exists():
            return candidate
    return start


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
os.chdir(PROJECT_ROOT)
print('PROJECT_ROOT =', PROJECT_ROOT)


In [ ]:
from dataclasses import dataclass, field
from typing import Any


@dataclass
class Step:
    step_id: str
    tool: str
    tool_input: str
    expected_output: str


@dataclass
class AgentState:
    topic: str
    steps: list[Step] = field(default_factory=list)
    findings: list[dict[str, Any]] = field(default_factory=list)
    errors: list[str] = field(default_factory=list)


In [ ]:
ALLOWED_TOOLS = {'web_search', 'calculator'}


def validate_step(step: Step) -> tuple[bool, str]:
    if not step.step_id.strip():
        return False, 'empty_step_id'
    if step.tool not in ALLOWED_TOOLS:
        return False, f'unsupported_tool:{step.tool}'
    if not step.tool_input.strip():
        return False, 'empty_tool_input'
    return True, 'ok'


def validate_state(state: AgentState) -> tuple[bool, list[str]]:
    issues = []
    if not state.topic.strip():
        issues.append('empty_topic')
    for s in state.steps:
        ok, reason = validate_step(s)
        if not ok:
            issues.append(f'{s.step_id}:{reason}')
    return len(issues) == 0, issues


In [ ]:
valid_state = AgentState(
    topic='AI planning market',
    steps=[
        Step('step_1', 'web_search', 'AI planning market 2026', 'market size and growth'),
        Step('step_2', 'calculator', '(10.9/3.66 - 1) * 100', 'growth percentage'),
    ],
)

invalid_state = AgentState(
    topic=' ',
    steps=[Step('', 'unknown', '', 'x')],
)

print('valid_state =>', validate_state(valid_state))
print('invalid_state =>', validate_state(invalid_state))

assert validate_state(valid_state)[0] is True
assert validate_state(invalid_state)[0] is False
print('Schema validation checks passed.')
